# Tâche 2 - Comme le disait le proverbe - les bons mots pour le dire

L'objectif de cette tâche est de compléter des proverbes à l'aide de modèles de langue N-grammes en insérant des mots aux bons endroits dans un texte masqué. Il s'agit d'une tâche de type *cloze test* qui consiste à choisir le meilleur mot à insérer dans un texte en fonction de son contexte. 

Voir l'énoncé du travail #1 pour une description plus détaillée de cette tâche. 

Fichiers:
- *proverbes.txt*: il contient plus de 3000 proverbes, un par ligne de texte. Pour l'entraînement des modèles de langues N-grammes. 
- *test_proverbes_v1.json*: il contient des proverbes masqués, les mots à insérer et la bonne formulation du proverbe. À utiliser pour évaluer la capacité des modèles à mettre les mots aux bons endroits. 

Consignes: 
- Utilisez NLTK pour construire les modèles de langue.
- Utilisez des expressions régulières (une seule ou plusieurs) pour remplacer les * et les ** par des mots. 
- Utilisez NLTK pour faire la tokenisation des proverbes. 
- N'oubliez pas de faire le *padding* des proverbes avec des symboles de début \<BOS\> et de fin \<EOS\>.
- Ne pas modifier les fonctions *load_proverbs* et *load_tests*.
- Ne pas modifier la signature de la fonction *replace_stars_with_words*.
- Utilisez la variable *models* pour conserver les modèles après entraînement. 
- Ne pas modifier la signature de la fonction *train_models*.
- Ne pas modifier la signature de la fonction *fill_masked_proverb*. 
- Des modifications aux signatures pourraient entraîner des déductions dans la correction. 
- Vous pouvez ajouter des cellules au notebook et toutes les fonctions utilitaires que vous voulez. 

## Section 1 - Lecture des fichiers de données (proverbes et tests)

In [1]:
import json

# Ne pas modifier le chemin de ces 2 fichiers pour faciliter notre correction
proverbs_fn = "./data/proverbes.txt"    
test_v1_fn = './data/test_proverbes_v1.json'

def load_proverbs(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        raw_lines = f.readlines()
    return [x.strip() for x in raw_lines]


def load_tests(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        test_data = json.load(fp)
    return test_data

In [2]:
proverbs = load_proverbs(proverbs_fn)

In [3]:
print("Nombre de proverbes pour l'entraînement: {}".format(len(proverbs)))
print("Un exemple de proverbe: " + proverbs[5])

Nombre de proverbes pour l'entraînement: 3108
Un exemple de proverbe: accord vaut mieux qu’argent


In [4]:
tests = load_tests(test_v1_fn)

In [5]:
import pandas as pd

def get_dataframe(test_proverbs):
    return pd.DataFrame.from_dict(test_proverbs, orient='columns', dtype=None, columns=None)

df = get_dataframe(tests)
df

,Masked,Word_list,Proverb
0,a beau * qui ** de loin,"[vient, mentir]",a beau mentir qui vient de loin
1,a * mentir qui vient de **,"[beau, loin]",a beau mentir qui vient de loin
2,l’* fait le **,"[larron, occasion]",l’occasion fait le larron
3,"*-toi, le ciel t’**","[aidera, aide]","aide-toi, le ciel t’aidera"
4,"année de *, ** de blé","[année, gelée]","année de gelée, année de blé"
5,"après la *, le ** temps","[beau, pluie]","après la pluie, le beau temps"
6,"aux échecs, les * sont les plus près des **","[fous, rois]","aux échecs, les fous sont les plus près des rois"
7,"ce que * veut, ** le veut","[femme, dieu]","ce que femme veut, dieu le veut"
8,bien mal * ne ** jamais,"[profite, acquis]",bien mal acquis ne profite jamais
9,bon * ne querelle pas ses **,"[outils, ouvrier]",bon ouvrier ne querelle pas ses outils


## Section 2 - Code pour substituer les masques (étoiles) par des mots

Expliquez ici comment vous procédez pour remplacer les étoiles des proverbes masqués par des mots... N'oubliez pas qu'il faut faire usage d'expressions régulières (une ou plusieurs - au choix). 



In [6]:
import re

def replace_stars_with_words(masked, word1, word2):
    """Remplace les * par word1 et word2 dans cet ordre. Retourne le proverbe complet."""
    # votre code
    proverb = re.sub(r"\*\*", word2, masked)
    proverb = re.sub(r"\*", word1, proverb)
    return proverb  # Retourne le proverbe avec des mots à la place des étoiles

## Section 3 - Construction des modèles de langue N-grammes. 

La fonction ***train_models*** prend en entrée une liste de proverbes et construit les trois modèles unigramme, bigramme et trigramme.

Les 3 modèles entraînés sont conservés dans *models*, un dictionnaire Python qui prend la forme 

<pre>
{
   1: modele_unigramme, 
   2: modele_bigramme, 
   3: modele_trigramme
}
</pre>

avec comme clé la valeur N du modèle et comme valeur le modèle construit par NLTK.

Expliquez ici comment vous procéder pour construire vos modèles avec NLTK, pour obtenir les n-grammes de mots, pour déterminer le vocabulaire, etc...



In [11]:
import nltk
from nltk.util import ngrams, pad_sequence
from nltk import word_tokenize
from nltk.lm.models import Laplace
from nltk import word_tokenize, bigrams, trigrams


# autres librairies à importer pour la partie sur les N-grammes?

BOS = '<BOS>'  # Jeton de début de proverbe
EOS = '<EOS>'  # Jeton de fin de proverbe

models = train_models(proverbs)  # les modèles entraînés - [1: modele_unigramme, 2: modele_bigramme, 3: modele_trigramme] 

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Utilisateur/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\Utilisateur\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [8]:
def build_vocabulary(text_list):
    all_unigrams = list()
    for sentence in text_list:
        word_list = word_tokenize(sentence.lower())
        all_unigrams = all_unigrams + word_list
    voc = set(all_unigrams)
    voc.add(BOS)
    voc.add(EOS)
    return list(voc)


In [9]:

def get_ngrams(text_list, n=2):
    all_ngrams = list()
    for sentence in text_list:
        tokens = word_tokenize(sentence.lower())
        padded_sent = list(pad_sequence(tokens, pad_left=True, left_pad_symbol=BOS, pad_right=True, right_pad_symbol=EOS, n=n))
        all_ngrams = all_ngrams + list(ngrams(padded_sent, n=n))      
    return all_ngrams

In [10]:
def train_models(proverbs):
    """ Vous ajoutez à partir d'ici le code dont vous avez besoin
        pour construire les différents modèles N-grammes.
        Cette fonction doit construire tous les modèles en une seule passe.
        Voir les consignes de l'énoncé du travail pratique concernant les modèles à entraîner.

        Vous pouvez ajouter les fonctions/méthodes et variables que vous jugerez nécessaire.
        Merci de ne pas modifier la signature et le comportement de cette fonction (nom, arguments, sauvegarde des modèles).
    """

    # Votre code à partir d'ici...  
    vocabulary = build_vocabulary(proverbs)
    
    unigram = get_ngrams(proverbs, n=1)
    bigram = get_ngrams(proverbs, n=2)
    trigram = get_ngrams(proverbs, n=3)
    #print(bigram)

    model1 = Laplace(1)
    model1.fit([unigram], vocabulary_text=vocabulary)
    
    model2 = Laplace(2)
    model2.fit([bigram], vocabulary_text=vocabulary)
    
    model3 = Laplace(3)
    model3.fit([trigram], vocabulary_text=vocabulary)

    
    # Sauvegarde de vos modèles 
    models = {
        1: model1,
        2: model2,
        3: model3
    }
    return models
    

## Section 4 - Compléter un proverbe

In [ ]:
# def fill_masked_proverb(masked, word_list, n=3, criteria="perplexity"):
#     """ Fonction qui complète un texte à trous (des mots masqués) en ajoutant 
#         les bons mots aux bons endroits (un "cloze test").

#         Le paramètre criteria indique la mesure qu'on utilise 
#         pour choisir le mot le plus approprié: "logprob" ou "perplexity".
#         On retourne l'estimation de cette mesure sur le proverbe complet,
#         c.-à-d. en utilisant tous les mots du proverbe.

#         Le paramètre n désigne le modèle utilisé.
#         1 - unigramme NLTK, 2 - bigramme NLTK, 3 - trigramme NLTK
        
#         Cette fonction retourne la solution (le proverbe complété) et 
#         la valeur de logprob ou perplexité (selon le paramètre en entrée de la fonction). 
#     """

#     # Votre code à partir d'ici. Vous pouvez modifier comme bon vous semble.
#     result = replace_stars_with_words(masked, word_list[0], word_list[1]).split()
#     result2 = replace_stars_with_words(masked, word_list[1], word_list[0]).split()

#     display(models[n].perplexity(result))
#     display(models[n].logscore(result))
#     display(models[n].perplexity(result2))
#     display(models[n].logscore(result2))

#     #---- À retirer à partir d'ici ----------
#     best_perplexity = 1500
#     best_logprob = -100
#     result = 'bonne femme fait le bon homme'
#     #---- À retirer ce qui précède ----------

#     if criteria == "perplexity":
#         score = best_perplexity
#     else:
#         score = best_logprob
#     return result, score

New code

In [ ]:
from itertools import permutations

def fill_masked_proverb(masked, word_list, n=3, criteria="perplexity"):
    
    #firstPartIndex = masked.split().index('*')
    #print(firstPartIndex)
    
    # Tokenize the masked proverb
    proverb_tokens = nltk.word_tokenize(masked)

    # Create the N-gram model based on the specified 'n'
    ngram_model = models[n]

    # Initialize variables to store the best result and score
    best_result = ""
    best_score = float('inf') if criteria == "perplexity" else float('-inf')

    # Generate all possible word permutations (4 combinations)
    word_permutations = list(permutations(word_list, 2))

    # Iterate through each word permutation
    for word1, word2 in word_permutations:
        # Replace the '*' in the proverb with the current words
        current_result = replace_stars_with_words(masked, word1, word2)
        completed_proverb = replace_stars_with_words(masked, word1, word2)
        
        
        # Calculate logprob or perplexity based on the chosen criteria
        if criteria == "perplexity":
            completed_proverb = get_ngrams([completed_proverb], n)
            score = ngram_model.perplexity(completed_proverb)
            # display("perplex of: ")
            # display(completed_proverb)
            # display(score)
            # print(current_result + str(score))
        else:
            #score = ngram_model.logscore(current_result.split()[-1],current_result.split()[:-1])
            score = ngram_model.logscore(completed_proverb)

        # Update the best result and score if the current score is better
        if (criteria == "perplexity" and score < best_score) or (criteria == "logprob" and score > best_score):
            best_score = score
            best_result = current_result

    return best_result, best_score

Un exemple pour illustrer l'utilisation de cette fonction

In [ ]:
masked =  "ami de * est bien **"   
#word_list = ['table', 'variable']    
word_list = [ 'variable','table']    
fill_masked_proverb(masked, word_list, n=3, criteria="logprob")

('ami de variable est bien table', -12.126704472843189)

## Section 5 - Expérimentations et analyse de vos résultats

Décrivez ici les résultats obtenus et présentez l'évaluation obtenue sur le fichier de test. Vous pouvez ajouter le nombre de cellules que vous souhaitez. 

In [ ]:
for test in tests:
    for i in range(1, 4):
        display(f"perplexity: {i} ->" + str(fill_masked_proverb(test['Masked'], test['Word_list'], n=i, criteria="perplexity")))
        display(f"logprob: {i} ->" + str(fill_masked_proverb(test['Masked'], test['Word_list'], n=i, criteria="logprob")))

"perplexity: 1 ->('a beau vient qui mentir de loin', 434.15031558678913)"

"logprob: 1 ->('a beau vient qui mentir de loin', -15.278993850906799)"

"perplexity: 2 ->('a beau mentir qui vient de loin', 1274.5300428347618)"

"logprob: 2 ->('a beau vient qui mentir de loin', -12.126704472843189)"

"perplexity: 3 ->('a beau mentir qui vient de loin', 1803.3372555720612)"

...

"logprob: 1 ->('quand la poire est tombe, elle mûre', -15.278993850906799)"

"perplexity: 2 ->('quand la poire est mûre, elle tombe', 1265.0539534717645)"

"logprob: 2 ->('quand la poire est tombe, elle mûre', -12.126704472843189)"

"perplexity: 3 ->('quand la poire est mûre, elle tombe', 1385.5482220915662)"

"logprob: 3 ->('quand la poire est tombe, elle mûre', -12.126704472843189)"

## Section 6 - Partie réservée pour faire nos tests lors de la correction

Merci de ne pas modifier ni retirer cette section du notebook ! 